In [151]:
from openai import OpenAI
orgID = %env CHAT_GPT_ORGANIZATION_ID
api_key = %env CHAT_GPT_API_KEY

client = OpenAI(
    api_key = api_key,
    organization=orgID,
)

In [152]:
from googleapiclient.discovery import build
import json
import re
from isodate import parse_duration
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd

api_key = %env YOUTUBE_SCAP_KEY
youtube = build('youtube', 'v3', developerKey=api_key)



In [174]:
channel_id = 'UCcefcZRL2oaA_uBNeo5UOWg' #ycombinator

def get_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        return transcript
    except Exception as e:
        print(f"An error occurred: {e}")
        return ''

video_id = 'hyYCn_kAngI'

tran = get_transcript(video_id)

tran

res = ''
res2 = ''
for text in tran:
    res += str(int(text['start'])) + '\n' + text['text'] + '\n'
    res2 += text['text'] + '\n'

res
tran

[{'text': 'foreign', 'start': 1.74, 'duration': 5.87}, {'text': '[Music]', 'start': 3.38, 'duration': 4.23}, {'text': "School my name is Gustav and I'm a group", 'start': 10.219, 'duration': 5.201}, {'text': "partner here at y combinator today I'm", 'start': 13.32, 'duration': 3.299}, {'text': 'going to talk about how to go from', 'start': 15.42, 'duration': 3.18}, {'text': 'talking to users', 'start': 16.619, 'duration': 4.08}, {'text': "getting your first customers here's what", 'start': 18.6, 'duration': 4.32}, {'text': 'I plan to cover today one what does it', 'start': 20.699, 'duration': 4.201}, {'text': "mean to do things that don't scale and", 'start': 22.92, 'duration': 4.439}, {'text': 'why is this mindset so so important at', 'start': 24.9, 'duration': 5.1}, {'text': 'this stage of your company two how to do', 'start': 27.359, 'duration': 4.861}, {'text': "sales I'll make the argument the founder", 'start': 30.0, 'duration': 4.02}, {'text': 'should be the ones doing sales in 

In [175]:

systemsText = """
You are a friendly and approachable expert in the startup domain, catering to a wide audience including those with varying levels of knowledge and experience. While maintaining a professional demeanor, your interactions should be warm and engaging, making complex startup concepts accessible and less intimidating. Stick to knowledge that is typically taught in educational settings, unless specifically asked to delve into more advanced or specialized topics. When providing answers, use simple language, prioritizing the 2000 most common words to ensure clarity and ease of understanding. This approach will make your advice more relatable and easier to grasp, especially for beginners. In situations requiring more advanced terminology or concepts not commonly taught in schools, clarify and simplify these terms as much as possible. Your primary goal is to educate and inspire aspiring entrepreneurs in a friendly and accessible manner, making the world of startups approachable for everyone.

You will be presented with the transcipt of a lecture. Your goal is to divide the talk into topics, where each topic ideally would be somewhere 1-5 minutes long. For each topic you should output the following:
1. Name of the topic
2. Very very small description of the topic
3. What second topic starts and what second topic ends 

Output as a json format
"""

# systemsText = """
# You are a friendly and approachable expert in the startup domain, catering to a wide audience including those with varying levels of knowledge and experience. While maintaining a professional demeanor, your interactions should be warm and engaging, making complex startup concepts accessible and less intimidating. Stick to knowledge that is typically taught in educational settings, unless specifically asked to delve into more advanced or specialized topics. When providing answers, use simple language, prioritizing the 2000 most common words to ensure clarity and ease of understanding. This approach will make your advice more relatable and easier to grasp, especially for beginners. In situations requiring more advanced terminology or concepts not commonly taught in schools, clarify and simplify these terms as much as possible. Your primary goal is to educate and inspire aspiring entrepreneurs in a friendly and accessible manner, making the world of startups approachable for everyone.
# """



mytext = str(res)


# systemsText = """
# You are a friendly and approachable expert in the startup domain, catering to a wide audience including those with varying levels of knowledge and experience. While maintaining a professional demeanor, your interactions should be warm and engaging, making complex startup concepts accessible and less intimidating. Stick to knowledge that is typically taught in educational settings, unless specifically asked to delve into more advanced or specialized topics. When providing answers, use simple language, prioritizing the 2000 most common words to ensure clarity and ease of understanding. This approach will make your advice more relatable and easier to grasp, especially for beginners. In situations requiring more advanced terminology or concepts not commonly taught in schools, clarify and simplify these terms as much as possible. Your primary goal is to educate and inspire aspiring entrepreneurs in a friendly and accessible manner, making the world of startups approachable for everyone.
# """


# mytext = """
# I will provide text with timestamps, please divide it into main topics. 
# Each topic should be 1-5 minutes long. 

# For each topic you should output the following:
# 1. Name of the topic
# 2. Very very small description of the topic
# 3. Timestamp the topic starts 

# Output as a json format.

# Here is the text: ```{}```""".format(res)
# mytext = """Please ignore all previous instructions. Please respond only in the English language. Do not explain what you are doing. Do not self reference. You are an expert text analyst and researcher. Please extract and name the main topics and keywords from the text I provide. Please present the results in a markdown table with name of topic, keywords related to main topic. Here is the text: ```{}```""".format(res)

# print(mytext)

In [176]:

resp = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {
          "role": "system",
            "content": systemsText
        },
        {
          "role": "user",
            "content": mytext
        }
    ],)

print(resp)

ChatCompletion(id='chatcmpl-8o5RmpjSl9BgUqr1isxd9M8AaPEFp', choices=[Choice(finish_reason='length', index=0, message=ChatCompletionMessage(content='{\n"1": {\n    "name": "Introduction",\n    "description": "Introducing the speaker and topic of the talk: How to go from talking to users to getting the first customers.",\n    "start_second": 1,\n    "end_second": 20\n},\n"2": {\n    "name": "Understanding the importance of doing things that don\'t scale",\n    "description": "Discussing the importance of doing things that don\'t scale by illustrating the journey of Airbnb as an example.",\n    "start_second": 20,\n    "end_second": 65\n},\n"3": {\n    "name": "Common misconceptions about startup growth",\n    "description": "Dispelling the misconception that a good product alone will lead to auto growth and explaining why founders must actively recruit customers.",\n    "start_second": 65,\n    "end_second": 97\n},\n"4": {\n    "name": "Missteps founders often make",\n    "description": 

In [178]:
print(resp.choices[0].message.content)
# topics = json.loads(resp.choices[0].message.content)

# topics 

{
"1": {
    "name": "Introduction",
    "description": "Introducing the speaker and topic of the talk: How to go from talking to users to getting the first customers.",
    "start_second": 1,
    "end_second": 20
},
"2": {
    "name": "Understanding the importance of doing things that don't scale",
    "description": "Discussing the importance of doing things that don't scale by illustrating the journey of Airbnb as an example.",
    "start_second": 20,
    "end_second": 65
},
"3": {
    "name": "Common misconceptions about startup growth",
    "description": "Dispelling the misconception that a good product alone will lead to auto growth and explaining why founders must actively recruit customers.",
    "start_second": 65,
    "end_second": 97
},
"4": {
    "name": "Missteps founders often make",
    "description": "Addressing common missteps founders make, like trying to replace customer interactions with building more product features.",
    "start_second": 97,
    "end_second": 11

In [172]:
def getTopic(topic, video_id):
    return {
        "name": topic["name"],
        "desctiption": topic["description"],
        "texts": ["text1.md"],
        "video_parts": [{
            "id": video_id,
            "start": topic["start_time"],
            "end": topic["end_time"]
        }],
        "node_type": "lecture",
        "parent_id": "startup-school",
        "thumbnail": ""
    }
startNum = 14
startChar = 'a'

res = {}

for topic in topics:
    print('"' + str(startNum) + startChar + '": ' + json.dumps(getTopic(topic, video_id), indent=4) + ',') 
    startChar = chr(ord(startChar) + 1)


"14a": {
    "name": "Re-Defining Launching",
    "desctiption": "A different perspective on the startup launching process, with advice on how not to overthink or overcomplicate the initial launch. Explanation on why constant iteration is more beneficial and how to adjust to failure or lack of initial traction.",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "u36A-YTxiOw",
            "start": 26,
            "end": 72
        }
    ],
    "node_type": "lecture",
    "parent_id": "startup-school",
    "thumbnail": ""
},
"14b": {
    "name": "When Should I Launch My Startup?",
    "desctiption": "Launch as soon as possible and iterate",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "u36A-YTxiOw",
            "start": 78,
            "end": 222
        }
    ],
    "node_type": "lecture",
    "parent_id": "startup-school",
    "thumbnail": ""
},
"14c": {
    "name": "Crafting A Clear One-Sentence Pitc

In [173]:
def overlaps(a1, b1, a2, b2):
    int1 = pd.Interval(a1, b1)
    int2 = pd.Interval(a2, b2)
    return int1.overlaps(int2)

def getTranscriptForTopic(topic, transcript):
    start = topic['start_time']
    end = topic['end_time']
    text = ''
    for snip in transcript:
        if overlaps(start, end, snip['start'], snip['start'] + snip['duration']):
            text+=snip['text']+' '
    return text


systemText = "You are a friendly and approachable expert in the startup domain, catering to a wide audience including those with varying levels of knowledge and experience. While maintaining a professional demeanor, your interactions should be warm and engaging, making complex startup concepts accessible and less intimidating. Stick to knowledge that is typically taught in educational settings, unless specifically asked to delve into more advanced or specialized topics. When providing answers, use simple language, prioritizing the 2000 most common words to ensure clarity and ease of understanding. This approach will make your advice more relatable and easier to grasp, especially for beginners. In situations requiring more advanced terminology or concepts not commonly taught in schools, clarify and simplify these terms as much as possible. Your primary goal is to educate and inspire aspiring entrepreneurs in a friendly and accessible manner, making the world of startups approachable for everyone."

for topic in topics:
    t = getTranscriptForTopic(topic, tran)
    text = """
Rewrite this from your own perspective, user simple words, write down as a lecture and use markdown to highlight
```
{}
```
""".format(t)
    
    resp = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
              "role": "system",
                "content": systemText
            },
            {
              "role": "user",
                "content": text
            }
        ],)
    print("====================================")
    print(resp.choices[0].message.content)
    print("====================================")


# _**The Art of Launching Your Startup**_

Hello there, today's chat is about helping businesses take that first big step: launching. I have a different viewpoint on this - one I hope will change your thoughts about launching. 

Lots of new business owners stress about their first launch. They think that it has to go perfectly. They believe this is their one and only chance for the public to see their product. They feel if their message isn't just right, customers won't buy their product, use it, or even invest in it. I chat with these owners all the time. They spend months painstakingly readying their launch.

But here's the tough reality: for most new businesses, the launch might happen, and it seems like nobody just cares. What's worse, if you spend half a year getting your product ready and no one is interested, your business might be in trouble before you even get another shot at launching.

So, I'd like to share a more flexible mindset. Just like the commonly known concept of "al

# Launch Like a Pro: Making Your Startup Pitch and Launch Successful

So, you want to come up with a great one-sentence pitch for your startup and launch it wisely. Don't worry, we've got your back. In the world of startups, continuous and strategic launch is one of the secrets of successful companies. 

## Why Launch Continuously?

It's all about learning. By launching your product early, even if it isn't perfect yet, you get to see what works, what doesn't, and how people react to your idea. You may even find out if you're dealing with the right users, because different communication channels will give you different audiences. Now let's dive into some types of launches you can explore.

## Understanding Different Types of Launches

### 1. The Silent Launch

This is a must and you can do it while you're at the planning stage. It doesn't need to be anything expensive. At this stage, all you need is:

- A domain name for your startup
- The name of your company
- A short description of w